In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
# Importing the Keras libraries and packages
import keras
from keras.models import Sequential
from keras.layers import Dense

In [3]:
# Importing the dataset
dataset = pd.read_csv('Churn_Modelling.csv')
X = dataset.iloc[:, 3:13].values
y = dataset.iloc[:, 13].values

In [4]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer

# Country column
ct = ColumnTransformer([("Transformer", OneHotEncoder(), [1,2])], remainder = 'passthrough')
X = ct.fit_transform(X)

In [5]:
# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [6]:
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [7]:
X.shape

(10000, 13)

In [8]:
# Initialising the ANN
classifier = Sequential()

# Adding the input layer and the first hidden layer
classifier.add(Dense(units = 6, kernel_initializer='glorot_uniform', activation = 'relu', input_dim = 13))

# Adding the second hidden layer
classifier.add(Dense(units = 6, kernel_initializer='glorot_uniform', activation = 'relu'))

# Adding the output layer
classifier.add(Dense(units = 1, kernel_initializer='glorot_uniform', activation = 'sigmoid'))

# Compiling the ANN
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

# Fitting the ANN to the Training set
classifier.fit(X_train, y_train, batch_size = 10, epochs = 100)


2022-05-12 10:26:21.939343: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Epoch 1/100
800/800 [==============================] - 1s 973us/step - loss: 0.5045 - accuracy: 0.7732
Epoch 2/100
800/800 [==============================] - 1s 925us/step - loss: 0.4316 - accuracy: 0.7960
Epoch 3/100
800/800 [==============================] - 1s 1ms/step - loss: 0.4136 - accuracy: 0.8098
Epoch 4/100
800/800 [==============================] - 1s 1ms/step - loss: 0.3960 - accuracy: 0.8304
Epoch 5/100
800/800 [==============================] - 1s 1ms/step - loss: 0.3748 - accuracy: 0.8470
Epoch 6/100
800/800 [==============================] - 1s 972us/step - loss: 0.3608 - accuracy: 0.8511
Epoch 7/100
800/800 [==============================] - 1s 907us/step - loss: 0.3560 - accuracy: 0.8529
Epoch 8/100
800/800 [==============================] - 1s 851us/step - loss: 0.3527 - accuracy: 0.8536
Epoch 9/100
800/800 [==============================] - 1s 823us/step - loss: 0.3508 - accuracy: 0.8561
Epoch 10/100
800/800 [==============================] - 1s 848us/step - loss: 0

800/800 [==============================] - 1s 922us/step - loss: 0.3344 - accuracy: 0.8641
Epoch 81/100
800/800 [==============================] - 2s 2ms/step - loss: 0.3340 - accuracy: 0.8643
Epoch 82/100
800/800 [==============================] - 1s 895us/step - loss: 0.3344 - accuracy: 0.8656
Epoch 83/100
800/800 [==============================] - 1s 828us/step - loss: 0.3341 - accuracy: 0.8650
Epoch 84/100
800/800 [==============================] - 1s 827us/step - loss: 0.3339 - accuracy: 0.8649
Epoch 85/100
800/800 [==============================] - 1s 825us/step - loss: 0.3338 - accuracy: 0.8644
Epoch 86/100
800/800 [==============================] - 1s 836us/step - loss: 0.3339 - accuracy: 0.8651
Epoch 87/100
800/800 [==============================] - 1s 922us/step - loss: 0.3341 - accuracy: 0.8640
Epoch 88/100
800/800 [==============================] - 1s 1ms/step - loss: 0.3340 - accuracy: 0.8641
Epoch 89/100
800/800 [==============================] - 1s 971us/step - loss: 0.3

In [22]:
# Predicting the Test set results
y_pred = classifier.predict(X_test)
y_pred = (y_pred > 0.5)
y_pred

array([[False],
       [False],
       [False],
       ...,
       [False],
       [False],
       [False]])

In [10]:
# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)

In [11]:
cm

array([[1512,   83],
       [ 197,  208]])

In [30]:
from aix360.algorithms.protodash import ProtodashExplainer

explainer = ProtodashExplainer()
m = 3
# call protodash explainer
# S contains indices of the selected prototypes
# W contains importance weights associated with the selected prototypes
(W, S, _) = explainer.explain(y_pred, y_train, m=m)



ValueError: shapes (8000,) and (1,2000) not aligned: 8000 (dim 0) != 1 (dim 0)

In [34]:
explainer = ProtodashExplainer()
(W, S, _) = explainer.explain(y_pred, y_pred, m=3)
dfs = dataset.iloc[S].copy()
dfs["Weight"] = W
print(dfs)
from aix360.algorithms.protodash import get_Gaussian_Data
# Gaussian (simulated) data example
# generate normalized gaussian data X, Y with 100 features and 300 & 4000 observations respectively
(X, Y) = get_Gaussian_Data(100, 300, 4000)
print(X.shape, Y.shape)
(W, S, _) = explainer.explain(X, Y, m=5, kernelType='Gaussian', sigma=2)
print(S, W)

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/aix360/algorithms/protodash/PDASH_utils.py:219: RuntimeWarning: invalid value encountered in true_divide
  w = np.max(u / K, 0)
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/cvxopt/coneprog.py:2111: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if 'x' in initvals:
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/cvxopt/coneprog.py:2116: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if 's' in initvals:
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/cvxopt/coneprog.py:2131: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if 'y' in initvals:
/Library/Frameworks/Python.framework/Vers

     pcost       dcost       gap    pres   dres
 0:  0.0000e+00 -2.0000e+04  4e+00  1e+00  0e+00
 1:  0.0000e+00 -4.9980e+11  5e+11  3e-16  5e-09
 2:  0.0000e+00 -4.9980e+09  5e+09  2e-16  1e-10
 3:  0.0000e+00 -4.9980e+07  5e+07  0e+00  0e+00
 4:  0.0000e+00 -4.9980e+05  5e+05  9e-17  2e-14
 5:  0.0000e+00 -4.9980e+03  5e+03  0e+00  4e-15
 6:  0.0000e+00 -4.9980e+01  5e+01  9e-17  4e-17
 7:  0.0000e+00 -4.9980e-01  5e-01  2e-16  3e-20
 8:  0.0000e+00 -4.9980e-03  5e-03  1e-16  4e-22
 9:  0.0000e+00 -4.9980e-05  5e-05  1e-16  4e-24
10:  0.0000e+00 -4.9980e-07  5e-07  9e-17  4e-26
11:  0.0000e+00 -4.9980e-09  5e-09  9e-17  4e-28
Optimal solution found.
     pcost       dcost       gap    pres   dres
 0:  0.0000e+00 -3.0000e+04  6e+00  1e+00  1e-01
 1:  6.3409e-01 -1.0425e+05  2e+01  1e+00  1e-01
 2:  5.3059e-01 -5.3416e+07  1e+04  1e+00  1e-01
 3:  3.8719e+07 -1.0976e+15  1e+15  4e-13  7e-06
 4:  3.8719e+07 -1.0977e+13  1e+13  3e-15  4e-06
 5:  3.8719e+07 -1.0993e+11  1e+11  2e-16  7e-0

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/cvxopt/coneprog.py:2111: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if 'x' in initvals:
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/cvxopt/coneprog.py:2116: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if 's' in initvals:
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/cvxopt/coneprog.py:2131: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if 'y' in initvals:
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/cvxopt/coneprog.py:2136: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if 'z' in initva

     pcost       dcost       gap    pres   dres
 0:  0.0000e+00 -2.0000e+04  4e+00  1e+00  1e+00
 1:  3.2221e+00 -8.5266e+04  2e+01  1e+00  1e+00
 2:  2.7001e-01 -1.6914e+07  4e+03  1e+00  1e+00
 3:  4.3897e+07 -3.9165e+14  4e+14  8e-13  8e-06
 4:  4.3897e+07 -3.9165e+12  4e+12  8e-15  5e-06
 5:  4.3894e+07 -3.9210e+10  4e+10  1e-16  5e-08
 6:  4.3518e+07 -4.3607e+08  5e+08  3e-16  6e-10
 7:  2.4531e+07 -2.8978e+07  5e+07  1e-16  7e-11
 8:  3.6567e+06 -5.7170e+06  9e+06  9e-17  2e-12
 9:  5.4550e+05 -5.8860e+05  1e+06  9e-17  6e-13
10:  7.8453e+04 -8.8545e+04  2e+05  4e-17  4e-13
11:  1.1222e+04 -1.2517e+04  2e+04  2e-16  1e-13
12:  1.5941e+03 -1.8062e+03  3e+03  2e-16  5e-14
13:  2.2267e+02 -2.6432e+02  5e+02  2e-16  1e-14
14:  2.9593e+01 -4.0124e+01  7e+01  8e-17  0e+00
15:  3.2462e+00 -6.6869e+00  1e+01  1e-16  9e-16
16: -3.8432e-02 -1.4066e+00  1e+00  3e-16  7e-16
17: -3.4476e-01 -4.9425e-01  1e-01  9e-17  3e-16
18: -3.5526e-01 -3.5919e-01  4e-03  2e-16  1e-16
19: -3.5527e-01 -3.55

In [35]:
print(S, W)

[3940 2539 2168 2189 1170] [0.20611975 0.24524152 0.19131865 0.17175151 0.16123794]


In [24]:
# Display the prototypes along with their computed weights
inc_prototypes = dataset.iloc[S, :].copy()
# Compute normalized importance weights for prototypes
inc_prototypes["Weights of Prototypes"] = np.around(W/np.sum(W), 2) 
inc_prototypes



,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Weights of Prototypes
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0,0.5
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1,0.5
